## *NOTE* Frequency indices T?90, R??mm, N??? have errors in them. Output files deleted.

Need to address the underlying function to count days above threshold per ensemble member.

This script generates the ETCCDI temperature indices from a single historical run of CESM2-WACCM daily temperature maxima (TX) and temperature minima (TN).

Outputs one file per index:

* TXX  
* TXN  
* TNN   
* TNX  
* TN90  
* TN10  
* TX90  
* TX10  
* SU   
* TR  
* ID 
* FD  

GSL Growing season length is not computed here. Script yet to be developed.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np # data arrays
import xarray as xr # data array manipulation
import pandas as pd
import datetime as dt
import os
import glob

Define the precipitation functions

In [3]:
from importlib import reload
import config
import tempex_func
import utils #pyclimdex by B Groenks


Read in each of the datasets in turn, calculate indices, then write the output to netcdf.

In [4]:
iDir = "/glade/work/maritye/Data/ARISE-MCB/Dailies/TREFHT/"
otnDir = "/glade/work/maritye/Data/ARISE-MCB/ETCCDI/TREFHTMN/"
otxDir = '/glade/work/maritye/Data/ARISE-MCB/ETCCDI/TREFHTMX/'

Create standard information to include with each data set

In [5]:
years = np.arange(2035,2070)
dates= '2035-2069'
member=range(10)
lat = np.linspace(-90,90,num=192)
lon = np.linspace(0,358.75,num=288)
dims = ('member','year', 'lat', 'lon')
coords = dict(member = member, year=years, lat=lat, lon=lon)
attribs = dict(description='Temperature Extreme Indices based on ETCCDI definitions. MCB scenarios with 5% global seeding', 
                history='Created by Mari Tye November 2023.' ),


read in thresholds from SSP245 take the mean of the ensemble as the threshold for these runs.

In [6]:
with xr.open_dataset('/glade/work/maritye/Data/ARISE-SAI/ETCCDI/SSP245/TREFHTMN/b.e21.BWSSP245cmip6.f09_g17.CMIP6-SSP2-4.5-WACCM.cam.h1.QN10.2015-2070.nc') as dsn10:
    dn10 = dsn10.mean(dim='members')

with xr.open_dataset('/glade/work/maritye/Data/ARISE-SAI/ETCCDI/SSP245/TREFHTMN/b.e21.BWSSP245cmip6.f09_g17.CMIP6-SSP2-4.5-WACCM.cam.h1.QN90.2015-2070.nc') as dsn90:
    dn90 = dsn90.mean(dim='members')
    
with xr.open_dataset('/glade/work/maritye/Data/ARISE-SAI/ETCCDI/SSP245/TREFHTMX/b.e21.BWSSP245cmip6.f09_g17.CMIP6-SSP2-4.5-WACCM.cam.h1.QX10.2015-2070.nc') as dsx10:
    dx10 = dsx10.mean(dim='members')

with xr.open_dataset('/glade/work/maritye/Data/ARISE-SAI/ETCCDI/SSP245/TREFHTMX/b.e21.BWSSP245cmip6.f09_g17.CMIP6-SSP2-4.5-WACCM.cam.h1.QX90.2015-2070.nc') as dsx90:
    dx90 = dsx90.mean(dim='members')

## Daily minimum temperatures

Create some blank arrays to fill with each index for each ensemble member. Write these out at the end with netcdf

In [8]:
TNX = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TNX')
TNN = xr.DataArray(None, coords=coords,  dims=dims, attrs=attribs, name='TNN')
TN90 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TN90')
TN10 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TN10')
TR = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TR')
FD = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='FD')

list the files - glob includes the filepath

In [9]:
files = glob.glob(iDir + '*.TREFHTMN.*')

In [10]:
for eff in range(1):
    print(eff)
    fp = files[eff]
    ds = xr.open_dataset(fp)
    ds = ds.sel(time=slice('2015-01-01', '2069-12-31'))

        ### Fixed index
    TNX[eff,:,:,:] = tempex_func.warmest_night(ds)    # Warmest night
    print('tnx')
    TNN[eff,:,:,:] = tempex_func.coolest_night(ds)    # Coolest night
    print('tnn')

        ## Fixed threshold count of days
    TR[eff,:,:,:] = tempex_func.tropical_nights(ds)   # Tropical Nights Tn>20C
    print('tr')
    FD[eff,:,:,:] = tempex_func.frost_days(ds) # Nights with TN<Oc
    print('fd')

    # Locally defined threshold
    TN90[eff,:,:,:] = tempex_func.annualnum_above_q(ds, dn90, varname='TREFHTMN')
    print('TN90')
    TN10[eff,:,:,:] = tempex_func.annualnum_below_q(ds, dn10, threshold=0.1, varname='TREFHTMN')
    print('TN10')
        

0
tnx
tnn
tr
fd
TN90
TN10


In [12]:
fileintro = 'b.e21.BSSP245smbb.f09_g17.MCB-050PCT.001-010.cam.h1.'

tnnnm = os.path.join(otnDir , (fileintro + 'TNN.' + dates+ ".nc"))
tnxnm = os.path.join(otnDir , (fileintro + 'TNX.' + dates+ ".nc"))
trnm = os.path.join(otnDir , (fileintro + 'TR.' + dates+ ".nc"))
fdnm = os.path.join(otnDir , (fileintro + 'FD.' + dates+ ".nc"))
tn90nm = os.path.join(otnDir , (fileintro + 'TN90.' + dates + ".nc"))
tn10nm = os.path.join(otnDir , (fileintro + 'TN10.' + dates+ ".nc"))

    
TNX = TNX.assign_attrs(units='Celsius',
                       longname = 'Warmest Night per year')
TNN = TNN.assign_attrs(units='Celsius',
                       longname = 'Coldest Night per year')
TR = TR.assign_attrs(units='Days per year',
                       longname = 'Tropical Nights; daily minimum above 20C')
FD = FD.assign_attrs(units='Days per year',
                       longname = 'Frost Days; daily minimum below 0C')
TN90 = TN90.assign_attrs(units='Days per year',
                       longname = 'Warm Nights above 90th Percentile')
TN10 = TN10.assign_attrs(units='Days per year',
                       longname = 'Cool Nights below 10th Percentile')


and write out to netcdf

In [13]:
TNN.to_netcdf(tnnnm)
TNX.to_netcdf(tnxnm)
TR.to_netcdf(trnm)
FD.to_netcdf(fdnm)
TN90.to_netcdf(tn90nm)
TN10.to_netcdf(tn10nm)


## Daily maximum temperature

In [14]:
TXX = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TXX')
TXN = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TXN')
TX10 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TX10')
TX90 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TX90')
SU = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='SU')
ID = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='ID')

In [15]:
files = glob.glob(iDir + '*.TREFHTMX.*')
for eff in range(10): 
    print(files[eff])
    #### Open file
    fp = files[eff]
    ds = xr.open_dataset(fp)
    ds = ds.sel(time=slice('2015-01-01', '2069-12-31'))

        ### Fixed thresholds
    TXX[eff,:,:,:] = tempex_func.warmest_day(ds)
    print('txx')
    TXN[eff,:,:,:] = tempex_func.coolest_day(ds)
    print('txn')

        ## Fixed threshold count of days
    SU[eff,:,:,:] = tempex_func.summer_days(ds) 
    print('su')
    ID[eff,:,:,:] = tempex_func.ice_days(ds) 
    print('id')
    
    #local threshold
    TX90[eff,:,:,:] = tempex_func.annualnum_above_q(ds, dx90, varname='TREFHTMX')
    print('TX90')
    TX10[eff,:,:,:] = tempex_func.annualnum_below_q(ds, dx10, threshold=0.1, varname='TREFHTMX')
    print('TX10')


/glade/work/maritye/Data/ARISE-MCB/Dailies/TREFHT/b.e21.BSSP245smbb.f09_g17.MCB-050PCT.007.cam.h1.TREFHTMX.20350101-20691231.nc
txx
txn
su
id
TX90
TX10
/glade/work/maritye/Data/ARISE-MCB/Dailies/TREFHT/b.e21.BSSP245smbb.f09_g17.MCB-050PCT.001.cam.h1.TREFHTMX.20350101-20691231.nc
txx
txn
su
id
TX90
TX10
/glade/work/maritye/Data/ARISE-MCB/Dailies/TREFHT/b.e21.BSSP245smbb.f09_g17.MCB-050PCT.010.cam.h1.TREFHTMX.20350101-20691231.nc
txx
txn
su
id
TX90
TX10
/glade/work/maritye/Data/ARISE-MCB/Dailies/TREFHT/b.e21.BSSP245smbb.f09_g17.MCB-050PCT.003.cam.h1.TREFHTMX.20350101-20691231.nc
txx
txn
su
id
TX90
TX10
/glade/work/maritye/Data/ARISE-MCB/Dailies/TREFHT/b.e21.BSSP245smbb.f09_g17.MCB-050PCT.002.cam.h1.TREFHTMX.20350101-20691231.nc
txx
txn
su
id
TX90
TX10
/glade/work/maritye/Data/ARISE-MCB/Dailies/TREFHT/b.e21.BSSP245smbb.f09_g17.MCB-050PCT.009.cam.h1.TREFHTMX.20350101-20691231.nc
txx
txn
su
id
TX90
TX10
/glade/work/maritye/Data/ARISE-MCB/Dailies/TREFHT/b.e21.BSSP245smbb.f09_g17.MCB-050PCT.0

In [22]:
txxnm = os.path.join(otxDir , (fileintro + 'TXX.' + dates + '.nc'))
txnnm = os.path.join(otxDir , (fileintro + 'TXN.' + dates + '.nc'))
sunm = os.path.join(otxDir , (fileintro + 'SU.' + dates + '.nc'))
idnm = os.path.join(otxDir , (fileintro + 'ID.' + dates + '.nc'))
tx90nm = os.path.join(otxDir , (fileintro + 'TX90.' + dates + '.nc'))
tx10nm = os.path.join(otxDir , (fileintro + 'TX10.' + dates + '.nc'))

TXX = TXX.assign_attrs(units='Celsius',
                       longname = 'Warmest Day per year')
TXN = TXN.assign_attrs(units='Celsius',
                       longname = 'Coldest Day per year')
SU = SU.assign_attrs(units='Days per year',
                       longname = 'Summer Days; daily maximum above 25C')
ID = ID.assign_attrs(units='Days per year',
                       longname = 'Ice Days; daily maximum below 0C')
TX90 = TX90.assign_attrs(units='Days per year',
                       longname = 'Warm Days above 90th Percentile')
TX10 = TX10.assign_attrs(units='Days per year',
                       longname = 'Cool Days below 10th Percentile')


    # and write out to netcdf
TXN.to_netcdf(txnnm)
TXX.to_netcdf(txxnm)
SU.to_netcdf(sunm)
ID.to_netcdf(idnm)
TX90.to_netcdf(tx90nm)
TX10.to_netcdf(tx10nm)
